In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys 
sys.path.insert(0, 'E:/repos/NYC_tax_demand_predictor')

In [3]:
from datetime import timedelta,datetime
from src.data import load_raw_data,transform_to_ts_data
import pandas as pd

In [4]:
current_time = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'current time {current_time}')

fetch_data_from = current_time- timedelta(days=28)
fetch_data_to = current_time

current time 2023-06-30 14:00:00


In [5]:
def fetch_data(
        data_from : datetime,
        data_to : datetime
)-> pd.DataFrame:

    fetch_data_from_ = data_from - timedelta(days=7*52)
    fetch_data_to_ = data_to - timedelta(days=7*52)

    rides = load_raw_data(fetch_data_from_.year,fetch_data_from_.month)
    rides = rides[rides['pu_datetime'] >= fetch_data_from_]
    rides2 = load_raw_data(fetch_data_to_.year,fetch_data_to_.month)
    rides2 = rides2[rides2['pu_datetime']<fetch_data_to_]

    rides = pd.concat([rides,rides2])
    rides['pu_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pu_location','pu_datetime'],inplace= True)

    return rides

In [7]:
historical_data = fetch_data(data_from=fetch_data_from,data_to=fetch_data_to)

file taxi_2022_06.parquet already downloaded.
file taxi_2022_07.parquet already downloaded.


In [8]:
historical_data.shape

(3303563, 2)

In [9]:
historical_data.head()

,pu_datetime,pu_location
292853,2023-06-02 14:40:47,1
289200,2023-06-02 14:55:04,1
289201,2023-06-02 15:00:54,1
297877,2023-06-02 15:42:58,1
307806,2023-06-02 16:10:04,1


In [10]:
transformed_data = transform_to_ts_data(historical_data)

100%|██████████| 261/261 [00:02<00:00, 88.56it/s] 


In [11]:
transformed_data.shape

(175392, 3)

In [12]:
transformed_data.head()

,pu_hour,rides,pu_location
0,2023-06-02 14:00:00,2,1
1,2023-06-02 15:00:00,2,1
2,2023-06-02 16:00:00,3,1
3,2023-06-02 17:00:00,3,1
4,2023-06-02 18:00:00,3,1


In [13]:
import hopsworks
import src.config as config

In [14]:
print(config.HOPSWORKS_PROJECT)
print(config.HOPSWORKS_API_KEY)

taxi_demand_salar
NNGm1wb8RgqjFJ0e.5Ef7P7UdYKlySV3YlaKItKCZtiablNzTFphIHQmcxVto66FU2QG4ycm3KGE636nT


In [15]:
project = hopsworks.login(
    project= config.HOPSWORKS_PROJECT,
    api_key_value= config.HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.2) by running 'pip install hopsworks==3.2.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/69786


In [16]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [17]:
feature_group = feature_store.get_or_create_feature_group(
    name = config.FEATURE_GROUP_NAME,
    version= config.FEATURE_GROUP_VERSION,
    description= 'Time series data with hourly frequency',
    primary_key= ['pu_location','pu_hour'],
    event_time='pu_hour'
)

In [18]:
feature_group.insert(transformed_data, write_options={'wait_for_job':False})

Uploading Dataframe: 0.00% |          | Rows 0/175392 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/69786/jobs/named/ts_data_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1e01ea6bb80>, None)